View .CSV file of expermints of Mill-Turn-Zollern-Motor

Load Libraries

In [ ]:
#### Load Libraries
import numpy as np
from joblib import load
import glob 
import  pandas as pd
import matplotlib.pyplot as plt
import os
from pymongo import MongoClient
import plotly.express as ex
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from datetime import datetime
from datetime import timedelta
import plotly.io as pio
from tensorflow.keras.models import Model, load_model

Set parameters

In [ ]:
version= 'v0.0.10'#'v0.0.8'
workspace= "G:\\Innovations@HELLER\\DN\\KI\\Zollern-FH-MillTrunMotor\\Datasets\\rough_data\\\database_with_sensor_synchronized\\Synchronization_results\\Versuch_19_20062023_ISO_Sync.csv"
assests_dir= 'G:\\Innovations@HELLER\\DN\\KI\\Zollern-FH-MillTrunMotor\\Datasets\\workspace\\assets\\'+version+'\\'
preprocessor_dir=assests_dir
preprocessor_name='preprocessor.p'
model_dir= assests_dir
model_name= 'model.h5'
###['Drehzahl','Temp_Motor','Temp_Klemmung', 'Temp_Lager']
model_input_Columns= ['I_MOMENT_G', 'T_LAGER', 'T_MOTOR', 'DRZ5']#['T_LAGER', 'T_MOTOR', 'T_KLEMMUNG']#['T_LAGER', 'T_MOTOR', 'T_KLEMMUNG']
output_variable= 'prediction'
prediction_variable= 'prediction'
window= 40#20v7#30v09
time_shift= +2
response_time= 5
time_relevance= 5
recording_time= 5
num_model_features= len(model_input_Columns)
smoothing_window= 10
to_smooth_signals='magnet_temperature'
target_variable= 'magnet_temperature_smoothed'
smoothing_window= 20

In [ ]:
def rounding(signals: np.ndarray):
    return np.round(signals,2)

In [ ]:
def get_abs_value(data):
    if type(data) is pd.DataFrame:
        result= data.to_numpy()
    else:    
        result= data
    result[:,-1]= np.abs(result[:,-1])
    return result

In [ ]:
### load models
pipeline= load(str(preprocessor_dir+preprocessor_name))
print('preprocessor is loaded successfully')
model= load_model(str(model_dir + model_name), compile= False)
print('Model is loaded successfully')
print(model.input_shape)

Simulate the prediction by passing values with 5 seconds

In [ ]:
def extract_time_relevance_array(input_array: np.ndarray, output_shape):
        input_time = input_array.shape[1]  # in case of 5 Seconds it will be 240 seconds
        output_time = output_shape[1]  # in case of 2 Minutes it will be 10
        step = int(input_time / output_time)
        result_array = np.zeros(output_shape)
        for i in range(0, output_time):
            #print('place in result: ', output_time - i - 1, ' place in original: ', input_time - i * step - 1)
            result_array[0, output_time - i - 1, :] = input_array[0, input_time - i * step - 1, :]
        return result_array

In [ ]:
def smoothing_data(input: pd.DataFrame, window= 240, signals=''):
    input[str(signals+'_smoothed')]= input[signals].rolling(window= window).mean().shift(-int(window/2)).round(2)
    input= input[window:-window]
    return input

In [ ]:
signals= pd.read_csv(workspace)
signals['date']= pd.to_datetime(signals['date'])
signals= smoothing_data(signals, window= smoothing_window, signals=to_smooth_signals)

In [ ]:
#### Simulate the prediction by passing values with 5 seconds ########
preds= []
input_data= signals.copy()
shape=(1,window,num_model_features)
output_sec_shape= (1, int(window*np.ceil((time_relevance/recording_time))), num_model_features)
print('Requested buffer shape to the model= {}', shape)
print('Orginal buffer shape to the model= {}', output_sec_shape)
first_run= True
inputsignal_list= model_input_Columns
last_prediction_time= None
out= 0
prev_record_time= None
for index, data_point in input_data.iterrows():
    if prev_record_time is not None and (data_point['date']  - prev_record_time).total_seconds() < recording_time:
        print('Escaped!')
        #preds.append(out)
        signals= signals[signals['date']!= data_point['date']]
        continue
    else:
        prev_record_time= data_point['date']
    #print ('index: ', index, ', Date: ', data_point['date'])
    new_signal_list = [float(data_point[sig]) for sig in inputsignal_list]
    #print ('New_signal_list: ', new_signal_list, ' timestamp: ', data_point['date'])
    rough_input_array= np.asarray([new_signal_list])
    signals_ = pipeline.transform(rough_input_array)#np.asarray([new_signal_list])#
    if first_run:
        #print ('index: ', index, ', Date: ', data_point['date'])
        #print('time_ diff= ', str((data_point['date'] - last_prediction_time).total_seconds()))
        # Auffüllen der Struktur - Bei erstem Start ist nur ein Datensatz vorhanden,
        # dieser wird mit np.full in alle kopiert
        hf_array= np.float32(np.full(output_sec_shape, signals_[0]))
        array= extract_time_relevance_array(hf_array,shape)
        #outputs = ort_sess.run(None, {'x': array})
        outputs =model.predict(array)
        #out = outputs[0][0][0] / 1000
        out = outputs[0][0]
        #print('Predciction First time. Input Buffer: ', array, 'prediction: ', out)
        preds.append(out)
        first_run = False
        last_prediction_time= data_point['date']
        #print('prediction_time: ', data_point['date'])
        continue
    elif (data_point['date'] - last_prediction_time).total_seconds()< response_time:
            #print('No prediction. Just shift')
            hf_array [0][:-1] = hf_array[0][1:]
            hf_array[0, -1, :] = signals_[0]
            preds.append(out)
            continue 
    else:
        #print ('index: ', index, ', Date: ', data_point['date'])
        #print('time_ diff= ', str((data_point['date'] - last_prediction_time).total_seconds()))
        #print("....Shifting the 5 seconds buffer and insert the new 5s-value")
        hf_array [0][:-1] = hf_array[0][1:]
        hf_array[0, -1, :] = signals_[0]
        array= extract_time_relevance_array(hf_array, shape)
        array = np.float32(array)
        ## Apply predicitions on 2 minutes buffer
        outputs =model.predict(array)
        #out = outputs[0][0][0] / 1000
        out = outputs[0][0]
        #print('Prediction Normal state. Input Buffer: ', array, 'prediction: ', out)
        preds.append(out)
        last_prediction_time= data_point['date']
        #print('prediction_time: ', data_point['date'])
    #signals['local_preds'][index]= out
    #print(signals.iloc[index])
    #print('#########################################################################')

In [ ]:
signals['local_preds']= preds
local_prediction_error= signals[output_variable] - signals['local_preds']

Visualize the results

In [ ]:
scatter_mode= 'lines'#'lines'# 'lines+markers'# 'markers'
selected_Columns= ['DRZ','Temp_Klemmung', 'Temp_Lager','Temp_Motor', prediction_variable, 'local_preds']
#print(preds)
fig= make_subplots(rows=6,cols=1,shared_xaxes= True, print_grid= True, vertical_spacing=0.02)
##DRZ
fig.add_trace(go.Scatter(x= signals['date'], y= signals['DRZ5'], name='given2model', mode= scatter_mode), row= 1, col= 1)
fig.update_yaxes(title_text= 'DRZ (RPM)', row= 1, col= 1)
##T_KLEMMUNG
fig.add_trace(go.Scatter(x= signals['date'], y= signals['T_KLEMMUNG'], name='T_KLEMMUNG', mode= scatter_mode), row= 2, col= 1)
fig.update_yaxes(title_text= 'T_KLEMMUNG C°', row= 2, col= 1)
##T_LAGER
fig.add_trace(go.Scatter(x= signals['date'], y= signals['T_LAGER'], name='T_LAGER', mode= scatter_mode), row= 3, col= 1)
fig.update_yaxes(title_text= 'T_LAGER C°', row= 3, col= 1)
##'T_MOTOR'
fig.add_trace(go.Scatter(x= signals['date'], y= signals['T_MOTOR'], name='T_MOTOR', mode= scatter_mode), row= 4, col= 1)
fig.update_yaxes(title_text= 'T_MOTOR C°', row= 4, col= 1)
##'prediction_variable'
#fig.add_trace(go.Scatter(x= signals['date'], y= signals[prediction_variable], name=prediction_variable, mode= scatter_mode), row= 5, col= 1)
fig.add_trace(go.Scatter(x= signals['date'], y= signals[target_variable], name=target_variable, mode= scatter_mode), row= 5, col= 1)
fig.add_trace(go.Scatter(x= signals['date'], y= signals['magnet_temperature'], name='magnet_temperature', mode= scatter_mode), row= 5, col= 1)
fig.add_trace(go.Scatter(x= signals['date'], y= signals['local_preds'], name='Datengetreibene Lösung', mode= scatter_mode), row= 5, col= 1)
fig.add_trace(go.Scatter(x= signals['date'], y= signals['prediction'], name='Machine_preds', mode= scatter_mode), row= 5, col= 1)
fig.add_trace(go.Scatter(x= signals['date'], y= signals['Analytic_sol'], name='Analytic_Lösung', mode= scatter_mode), row= 5, col= 1)
fig.update_yaxes(title_text= 'Rotor Temperatur C°', row= 5, col= 1)
# ##Restfehler
fig.add_trace(go.Scatter(x= signals['date'], y= signals['local_preds'] - signals['magnet_temperature'], name=' Restfehler datengetriebne Vorhersage', mode= scatter_mode),  row= 6 , col= 1)
fig.add_trace(go.Scatter(x= signals['date'], y= signals['Analytic_sol'] - signals['magnet_temperature'], name=' Restfehler analytische vorhersage', mode= scatter_mode),  row= 6 , col= 1)
# fig.add_trace(go.Scatter(x= signals['date'], y= signals['Analytic_sol'] - df['magnet_temperature'], name=' Restfehler analytische Lösung', mode= scatter_mode),  row= 6 , col= 1)
#     ## Draw the tolerence +-5
# #fig.add_trace(go.Scatter(x= signals['date'], y= np.full_like(signals['db_prediction_abs_error'],5), name='+5 Obere Grenze', mode= scatter_mode),  row= 10 , col= 1)
# #fig.add_trace(go.Scatter(x= signals['date'], y= np.full_like(signals['db_prediction_abs_error'],-5), name='- 5 Obere Grenze', mode= scatter_mode),  row= 10 , col= 1)
fig.update_yaxes(title_text= 'Fehler bei Rotor Temperatur C°', row= 6, col= 1)
fig.update_layout(height=1200, width=1400, title_text= 'M57002 Machine Data')
fig.show()